In [1]:
# Importando bibliotecas
import pprint
import pandas as pd
import copy

# Importando arquivos
from cycles import calculate_two_evaporators_cycle

In [4]:
def determine_if_reached_threshold(value, lower, upper):
    reached_threshold = 0
    if value <= lower:
        reached_threshold = 1
        value = lower
    elif value >= upper:
        reached_threshold = 1
        value = upper  
    return value, reached_threshold

def calculate_next_x(input_values, x, current_cycle, delta, alpha, lower_threshold, upper_threshold):
    current_x = input_values[x]
    input_values_x = copy.copy(input_values)
    input_values_x[x] += delta
    gradient_cycle_x = calculate_two_evaporators_cycle(input_values_x)
    gradient_x = (gradient_cycle_x['cop'] - current_cycle['cop']) / delta
    next_x = current_x + alpha * gradient_x
    next_x, reached_threshold_x = determine_if_reached_threshold(next_x, lower_threshold, upper_threshold)
    return next_x, reached_threshold_x

def calculate_next_all(input_values, delta, alpha):
      
    current_cycle = calculate_two_evaporators_cycle(input_values)

    # Next superheating_ht
    next_superheating_ht, reached_threshold_superheating_ht = calculate_next_x(input_values, 
                                                                               'superheating_ht', 
                                                                               current_cycle, 
                                                                               delta, 
                                                                               alpha, 
                                                                               input_values['lower_threshold'], 
                                                                               input_values['upper_threshold'])

    # Next superheating_lt
    next_superheating_lt, reached_threshold_superheating_lt = calculate_next_x(input_values, 
                                                                               'superheating_lt', 
                                                                               current_cycle, 
                                                                               delta, 
                                                                               alpha, 
                                                                               input_values['lower_threshold'], 
                                                                               input_values['upper_threshold'])

    # Next subcooling
    next_subcooling, reached_threshold_subcooling = calculate_next_x(input_values, 
                                                                     'subcooling', 
                                                                     current_cycle, 
                                                                     delta, 
                                                                     alpha, 
                                                                     input_values['lower_threshold'], 
                                                                     input_values['upper_threshold'])
        
    input_values['superheating_ht'] = next_superheating_ht
    input_values['superheating_lt'] = next_superheating_lt
    input_values['subcooling'] = next_subcooling
    sum_of_threshold_reached =  reached_threshold_superheating_ht + reached_threshold_superheating_lt \
        + reached_threshold_subcooling
    
    next_cycle = calculate_two_evaporators_cycle(input_values)
    
    return sum_of_threshold_reached, current_cycle, next_cycle

def optimize_with_cop(input_values):
    n = 0
    error = 1
    while n < 4 and abs(error) >= 10**(-10):
        n, current_cycle, next_cycle = calculate_next_all(input_values, 10**(-1), 5)
        error = (next_cycle['cop'] - current_cycle['cop'])/((next_cycle['cop'] + current_cycle['cop'])/2)
    optimized_cycle = calculate_two_evaporators_cycle(input_values)
    
    return optimized_cycle

def optimize_multiple_refrigerants(input_values, input_ranges):
    original_input_values = copy.copy(input_values)
    results = pd.DataFrame(columns=['refrigerant',
                                    'work',
                                    'q_evaporator_ht',
                                    'q_evaporator_lt',
                                    'subcooling',
                                    'superheating_ht',
                                    'superheating_lt',
                                    'annual_energy_consumption',
                                    'annual_price',
                                    'cop', 
                                    'exergy_efficiency'])
    n = 0
    for refrigerant in input_ranges['refrigerants']:
        n += 1
        print(n)
        input_values = copy.copy(original_input_values)
        input_values['refrigerant'] = refrigerant
        optimized_cycle = optimize_with_cop(input_values)
        results = results.append({
            'refrigerant': refrigerant,
            'work': optimized_cycle['cycle_inputs']['work'],
            'q_evaporator_ht': optimized_cycle['q_evaporator_ht'],
            'q_evaporator_lt': optimized_cycle['q_evaporator_lt'],
            'subcooling': optimized_cycle['cycle_inputs']['subcooling'],
            'superheating_ht': optimized_cycle['cycle_inputs']['superheating_ht'],
            'superheating_lt': optimized_cycle['cycle_inputs']['superheating_lt'],
            'annual_energy_consumption': optimized_cycle['cycle_inputs']['annual_energy_consumption'],
            'annual_price': optimized_cycle['cycle_inputs']['annual_price'],
            'cop': optimized_cycle['cop'],
            'exergy_efficiency': optimized_cycle['exergy_efficiency']
        }, ignore_index=True)
    print('Done')
    return results

In [5]:
# Temperatura externa: https://en.climate-data.org/south-america/brazil/rio-de-janeiro/rio-de-janeiro-853/ (media das maximas)
# Temperaturas internas: pagina 18 do arquivo MFL70720817_PB.pdf (LG)
# Refrigerante base: pagina 6 do arquivo MFL70720817_PB.pdf (LG)
# Consumo: pagina 16 do arquivo CATALOGO_10_BRASTEMP (Brastemp)
# Tarifa media do kWh: https://www.aneel.gov.br/ranking-das-tarifas (Light RJ)
input_values = {
    't_external_env': 28.4 + 273.15,
    't_internal_env_ht': 3 + 273.15,
    't_internal_env_lt': -18 + 273.15,
    'approach_condenser': 5,
    'approach_evaporator_ht': 5,
    'approach_evaporator_lt': 5,
    'work': (42 * 1000) / (30 * 24),
    'annual_energy_consumption': 42 * 12,
    'annual_price': 42 * 12 * 0.694,
    'f': 0.5,
    'isentropic_efficiency': 0.7,
    'subcooling': 5,
    'superheating_ht': 5,
    'superheating_lt': 5,
    'upper_threshold': 10,
    'lower_threshold': 0
}

input_ranges = {
    'refrigerants': ['R600a', 'R134a', 'R22', 'R290', 'R1234yf', 'NH3']
}

optimized_table = optimize_multiple_refrigerants(input_values, input_ranges)
optimized_table.to_excel(r'C:\repositories\projeto-final\src\main_optimization.xlsx', index = False)

1
2
3
4
5
6
Done
